In [ ]:
# %matplotlib inline
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from espm.models import edxs
from espm.models.EDXS_function import print_concentrations_from_W
from espm.conf import DATASETS_PATH
from pathlib import Path
from espm.datasets import spim
import espm.tables_utils as tu
from espm.estimators import SmoothNMF

In [ ]:
s = hs.load("/home/adrien/travail/espm/generated_datasets/FpBrgCaPv_N293_paper/sample_0.hspy")
s.change_dtype("float64")

In [ ]:
s.metadata.Sample.thickness = 0.0

In [ ]:
G = s.build_G(norm = True)
est = SmoothNMF( n_components = 3,tol=0.00001, max_iter = 100, G = G,hspy_comp = True)

s.decomposition(algorithm=est)

In [ ]:
x = np.linspace(s.axes_manager[2].offset,s.axes_manager[2].offset + s.axes_manager[2].scale*s.axes_manager[2].size, num = s.axes_manager[2].size)
plt.plot(x,G()[:,10])

In [ ]:
s.metadata.Sample.elements

In [ ]:
W = est.W_
H = est.H_
fe1 = (W[10]@H).reshape((128,128))
fe2 = (W[11]@H).reshape((128,128))



In [ ]:
from espm.models.absorption_edxs import absorption_correction, absorption_coefficient
A = absorption_correction(0.7,thickness=0.1, density=3.0, toa=90,elements_dict = {"26" : 1.0})

In [ ]:
energy = 0.7
toa = s.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle

def abs_map(l_map, energy, toa, mass_thickness, elements_dict = {"26" :1.0}) : 
    mu = absorption_coefficient(energy,atomic_fraction=False,elements_dict= elements_dict)
    rad_toa = np.deg2rad(toa)
    chi = mu*mass_thickness/np.sin(rad_toa)
    return l_map*((1 - np.exp(-chi))/chi)


m = abs_map(fe1,energy,toa,np.random.rand(128,128))

plt.imshow(m)

In [ ]:
# <examples/doc_parameters_basic.py>
import numpy as np

from lmfit import Minimizer, Parameters, report_fit
from tqdm import tqdm

# define objective function: returns the array to be minimized
def absorption_fit(params,data,energy1, energy2, toa, elements_dict = {}):
    """Model a decaying sine wave and subtract data."""
    mt = params['mass_thickness']
    mu1 = absorption_coefficient(energy1,atomic_fraction=False,elements_dict= elements_dict)
    mu2 = absorption_coefficient(energy2,atomic_fraction=False,elements_dict= elements_dict)
    rad_toa = np.deg2rad(toa)
    chi1 = mu1*mt/np.sin(rad_toa)
    chi2 = mu2*mt/np.sin(rad_toa)
    model = (1 - np.exp(-chi2))/(1 - np.exp(-chi1))*(mu1/mu2)
    return model - data


# create a set of Parameters
params = Parameters()
params.add('mass_thickness', value=0.0085, min=1e-10)

# do fit, here with the default leastsq algorithm
mt_map = np.zeros_like(s.data[:,:,0])
data = fe1/fe2
for i in tqdm(range(mt_map.shape[0])) : 
    for j in range(mt_map.shape[1]) :
        curr_data = data[i,j]
        minner = Minimizer(absorption_fit, params, fcn_args=(curr_data,0.7,6.4,s.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, {'26' : 1.0} ))
        result = minner.minimize()
        mt_map[i,j] = result.params['mass_thickness'].value
        params['mass_thickness'].value = result.params['mass_thickness'].value

In [ ]:
def absorption_mass_thickness(x,mass_thickness, toa = 90, atomic_fraction = True, *, elements_dict = {"Si" : 1.0}) : 
    mu = absorption_coefficient(x,atomic_fraction,elements_dict = elements_dict)
    rad_toa = np.deg2rad(toa)
    if type(mu) == float : 
        chi = mu*mass_thickness/np.sin(rad_toa)
    else : 
        chi = mu[:,np.newaxis]@((mass_thickness.reshape(-1)[:,np.newaxis]).T)/np.sin(rad_toa)
    return (1-np.exp(-chi))/chi

In [ ]:
ed

In [ ]:
plt.plot(s.data.sum(axis=(0,1)))

In [ ]:
ed = {}
i = 0
for elt in s.metadata.Sample.elements : 
    if elt == "Fe" : 
        i += 1
    ed[elt] = np.mean(est.W_[i])
    i+=1

In [ ]:
absc = absorption_mass_thickness(x,mt_map,toa = s.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, elements_dict = ed)

In [ ]:
abscr = absc.T.reshape((128,128,1980))

In [ ]:
sabscr = hs.signals.Signal1D(abscr)

In [ ]:
sabscr.plot()

In [ ]:
s.data = s.data/sabscr.data

In [ ]:
sabscr.plot()

In [ ]:
G = s.build_G(norm = True)
est = SmoothNMF( n_components = 3,tol=0.00001, max_iter = 100, G = G,hspy_comp = True)

s.decomposition(algorithm=est)

In [ ]:
s.plot_decomposition_results()

In [ ]:
spim = hs.load("eds_rebin.hspy")

spim.set_signal_type("EDS_espm")
spim.set_additional_parameters(thickness = 0.0,density = 2.1,detector_type = "Det_eff.txt",width_slope = 0.01,width_intercept = 0.065,xray_db = "300keV_corentin_xrays.json")
spim.metadata.Sample.elements = ['Al','C','Cr','Cu','F','Fe','Mg','N','Na','Ni','O','S','Si']

# spim.set_analysis_parameters(beam_energy = 300,
# azimuth_angle = 0.0,
# elevation_angle = 35.0,
# elements = ['Al', 'C', 'Ca', 'Cl', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ni', 'O', 'P', 'Pt', 'S', 'Si', 'Ti', 'Zr'],
# tilt_stage = -6.071,
# thickness = 80e-7,
# density = 2.1,
# detector_type = "Det_eff.txt",
# width_slope = 0.01,
# width_intercept = 0.065,
# xray_db = "300keV_corentin_xrays.json")

cropped_spim = spim.inav[1:195,:].isig[:7.8]

In [ ]:
spectrum = hs.load("phyllo_spectre_moyen.hspy")
spectrum.set_signal_type("EDS_espm")
spectrum.axes_manager[0].offset += +0.000
spectrum.set_additional_parameters(thickness = 0.0,density = 2.1,detector_type = "Det_eff.txt",width_slope = 0.01,width_intercept = 0.065,xray_db = "300keV_corentin_xrays.json")
spectrum.metadata.Sample.elements = ['K','Pt','Al','C','Cr','Cu','F','Fe','Ga','Mg','N','Na','Ni','O','S','Si']

cropped_spim = spectrum.isig[:7.8]

G_ref = cropped_spim.build_G("bremsstrahlung", norm = True, reference_elt={'26' : 3.0,"24" : 3.0})

est_ref = SmoothNMF(n_components = 1, tol = 0.000001, max_iter = 10000000, G = G_ref)
 
est_ref.fit(cropped_spim.data[:,np.newaxis])

fW = est_ref.W_
fG = est_ref.G_

x = np.linspace(cropped_spim.axes_manager[0].offset, cropped_spim.axes_manager[0].offset + cropped_spim.axes_manager[0].scale*cropped_spim.axes_manager[0].size, num = cropped_spim.axes_manager[0].size)

# <examples/doc_parameters_basic.py>
import numpy as np

from lmfit import Minimizer, Parameters, report_fit
from tqdm import tqdm
from espm.models.absorption_edxs import absorption_correction, absorption_coefficient

# define objective function: returns the array to be minimized
def absorption_fit(params,data,energy1, energy2, toa, elements_dict = {}):
    """Model a decaying sine wave and subtract data."""
    mt = params['mass_thickness']
    mu1 = absorption_coefficient(energy1,atomic_fraction=False,elements_dict= elements_dict)
    mu2 = absorption_coefficient(energy2,atomic_fraction=False,elements_dict= elements_dict)
    rad_toa = np.deg2rad(toa)
    chi1 = mu1*mt/np.sin(rad_toa)
    chi2 = mu2*mt/np.sin(rad_toa)
    model = (1 - np.exp(-chi1))/(1 - np.exp(-chi2))*(mu2/mu1)
    return model - data


# create a set of Parameters
params = Parameters()
params.add('mass_thickness', value=0.0005, min=1e-10)


cr_ratio = fW[2]/fW[3]
fe_ratio = fW[6]/fW[7]
# do fit, here with the default leastsq algorithm


curr_data = fe_ratio
minner = Minimizer(absorption_fit, params, fcn_args=(curr_data,0.7,6.4,cropped_spim.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, {"26" : 1.0} ))
result = minner.minimize()
print(result.params['mass_thickness'].value)
        # params['mass_thickness'].value = result.params['mass_thickness'].value

plt.plot(x,cropped_spim.data,'ko',markersize = 3.5,label = "Experimental")
plt.plot(x,fG@fW,'r',linewidth = 2, label = "Simulated")
plt.legend()
plt.xlabel("Energy (keV)")
plt.ylabel("Intensity (cts)")
plt.ylim(0,4)
plt.tight_layout()

In [ ]:
spectrum = hs.load("phyllo_spectre_moyen.hspy")
spectrum.set_signal_type("EDS_espm")
spectrum.axes_manager[0].offset += +0.000
spectrum.set_additional_parameters(thickness = 0.0004934,density = 1.0,detector_type = "Det_eff.txt",width_slope = 0.01,width_intercept = 0.065,xray_db = "300keV_corentin_xrays.json")
spectrum.metadata.Sample.elements = ['Al','C','Cr','Cu','F','Fe','Ga','Mg','N','Na','Ni','O','S','Si']

cropped_spim = spectrum.isig[:7.8]

G_nref = cropped_spim.build_G("bremsstrahlung", norm = True, reference_elt={})

est_nref = SmoothNMF(n_components = 1, tol = 0.000001, max_iter = 10000000, G = G_nref)
 
est_nref.fit(cropped_spim.data[:,np.newaxis])

fWn = est_nref.W_
fGn = est_nref.G_

In [ ]:
W = np.vstack((fW[:2], fW[3:6],fW[7:]))
nW = W/W.sum(axis = 0,keepdims = True)
siW = nW/nW[-3]


i = 0
for j in cropped_spim.metadata.Sample.elements : 
    if j == 'Cu' : 
        pass
    else : 
        print("{} : {}".format(j,siW[i]))
        i +=1

In [ ]:
fW/fW[-3]

In [ ]:
fWn/fWn[-3]

In [ ]:
G_no_ref = cropped_spim.build_G("bremsstrahlung",norm = True, reference_elt = {})
est_no_ref = SmoothNMF( n_components = 7,tol=0.000001, max_iter = 1000000, G = G_no_ref,hspy_comp = True)

cropped_spim.decomposition(algorithm=est_no_ref)

In [ ]:
cropped_spim.save("eds_rebin_results_no_thickness_noCu.hspy")

In [ ]:
G_ref = cropped_spim.build_G("bremsstrahlung",norm = True, reference_elt = {"26":3.0})
est_ref = SmoothNMF( n_components = 7,tol=0.000001, max_iter = 1000000, G = G_ref,hspy_comp = True)

cropped_spim.decomposition(algorithm=est_ref)

In [ ]:
W = est_ref.W_
H = est_ref.H_

iron_ratio = ((W[5]@H).reshape((68,194)))/((W[6]@H).reshape((68,194)))

# iron_ratio = (W[6]@H).reshape((68,195))/(W[7]@H).reshape((68,195)) 

In [ ]:
ed = {}
i = 0
W = est_ref.W_
for elt in cropped_spim.metadata.Sample.elements : 
    if elt == "Fe" : 
        i += 1
    ed[elt] = np.mean(W[i])
    i+=1

ed

In [ ]:
# <examples/doc_parameters_basic.py>
import numpy as np

from lmfit import Minimizer, Parameters, report_fit
from tqdm import tqdm
from espm.models.absorption_edxs import absorption_correction, absorption_coefficient

# define objective function: returns the array to be minimized
def absorption_fit(params,data,energy1, energy2, toa, elements_dict = {}):
    """Model a decaying sine wave and subtract data."""
    mt = params['mass_thickness']
    mu1 = absorption_coefficient(energy1,atomic_fraction=False,elements_dict= elements_dict)
    mu2 = absorption_coefficient(energy2,atomic_fraction=False,elements_dict= elements_dict)
    rad_toa = np.deg2rad(toa)
    chi1 = mu1*mt/np.sin(rad_toa)
    chi2 = mu2*mt/np.sin(rad_toa)
    model = (1 - np.exp(-chi1))/(1 - np.exp(-chi2))*(mu2/mu1)
    return model - data


# create a set of Parameters
params = Parameters()
params.add('mass_thickness', value=0.00085, min=1e-10)

# do fit, here with the default leastsq algorithm
mt_map = np.zeros_like(cropped_spim.data[:,:,0])
for i in tqdm(range(mt_map.shape[0])) : 
    for j in range(mt_map.shape[1]) :
        curr_data = iron_ratio[i,j]
        minner = Minimizer(absorption_fit, params, fcn_args=(curr_data,0.7,6.4,cropped_spim.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, {"26" : 1.0} ))
        result = minner.minimize()
        mt_map[i,j] = result.params['mass_thickness'].value
        # params['mass_thickness'].value = result.params['mass_thickness'].value

In [ ]:
plt.imshow(mt_map)

In [ ]:
offset = cropped_spim.axes_manager[2].offset
size = cropped_spim.axes_manager[2].size
scale = cropped_spim.axes_manager[2].scale
x = np.linspace(offset, scale*size + offset, num = size)
absc = absorption_mass_thickness(x,mt_map,toa = cropped_spim.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, elements_dict = {"F" : 0.47,"Fe" : 0.25, "Si" : 1.0, "O" : 2.6, "Mg" : 0.9, "C" : 0.6})
abscr = absc.T.reshape((cropped_spim.axes_manager[1].size,cropped_spim.axes_manager[0].size,size))

In [ ]:
hs.signals.Signal1D(abscr).plot()

In [ ]:
plt.plot(x,abs_corr.get_decomposition_model().mean().data, label = 'abs corr')
plt.plot(x,cropped_spim.mean().data,'ko', markersize = 1.3,  label = "exp")
plt.plot(x, no_abs.get_decomposition_model().mean().data, label = 'no thickness')
plt.plot(x,abs.get_decomposition_model().mean().data,label = "abs")
plt.legend()

In [ ]:
ns = cropped_spim.copy()
ns.data = cropped_spim.data/abscr

In [ ]:
cropped_spim.plot()

In [ ]:
G_aft = ns.build_G("bremsstrahlung",norm = True, reference_elt={})
est_aft = SmoothNMF( n_components = 7,tol=0.000001, max_iter = 1000000, G = G_aft,hspy_comp = True)

ns.decomposition(algorithm=est_aft)

In [ ]:
ns.save("eds_rebin_results_after_correction_noCu.hspy")

In [ ]:
W = np.vstack((est_aft.W_[:3], est_aft.W_[4:]))
nW = W/W.sum(axis = 0,keepdims = True)
sW = np.sum(nW,axis = 1)
siW = sW/sW[-3]

W1 = np.vstack((est_no_ref.W_[:5], est_no_ref.W_[6:]))
nW1 = W1/W1.sum(axis = 0,keepdims = True)
sW1 = np.sum(nW1,axis = 1)
siW1 = sW1/sW1[-5]

i = 0
for j in cropped_spim.metadata.Sample.elements : 
    if j == 'Cu' : 
        pass
    else : 
        print("{} : {}".format(j,siW[i]))
        i +=1

i = 0
for j in cropped_spim.metadata.Sample.elements : 
    if j == 'Cu' : 
        pass
    else : 
        print("{} : {}".format(j,siW1[i]))
        i +=1

In [ ]:
ns.plot_decomposition_results()

In [ ]:
cropped_spim.plot_decomposition_results()

In [ ]:
factors = ns.get_decomposition_factors().data
plt.plot(factors[4])

In [ ]:
H = est_aft.H_.reshape((7,68,194))
factors = est_ref.G_@est_ref.W_
for i,j in enumerate(H) : 
    new_mt = (mt_map*j).clip(min = 1e-10)
    abs_map = absorption_mass_thickness(x,new_mt,toa = cropped_spim.metadata.Acquisition_instrument.TEM.Detector.EDS.take_off_angle, elements_dict = ed)
    print(np.isnan(factors[i]).sum())
    factors[i] /= np.mean(abs_map)

In [ ]:
resu = (factors@est_ref.H_).sum(axis = 1 )/(194*68)

In [ ]:
no_abs = hs.load("eds_rebin_results_no_thickness_noCu.hspy")

In [ ]:
# plt.plot(x,abs.mean().data)
plt.plot(x+0.005,resu, label = 'abs_corr')
plt.plot(x,no_abs.get_decomposition_model().mean().data, label = "no")
plt.plot(x,cropped_spim.mean().data, 'ko', markersize = 1.5, label = "exp")
plt.legend()

In [ ]:
np.isnan(factors).sum()

In [ ]:
sp =  hs.load(filenames=["sp_C.hspy","sp_FeNiS.hspy","sp_FeO.hspy","sp_FeS.hspy","sp_Si_1.hspy","sp_Si_2.hspy","sp_ZnS.hspy"])
maps = hs.load("Map_mixing_model_7comp.hspy")

In [ ]:
maps.plot(title = "truth")
ns.get_decomposition_loadings().plot()

In [ ]:
fig, ax = plt.subplots()
ax.plot(sp[4]*10)
ax.plot(factors[:,0])

In [ ]:
fig1,ax1 = plt.subplots()
ax1.plot(x,sp[1])
ax1.plot(x,factors[2])

In [ ]:
fig2, ax2 = plt.subplots()
ax2.imshow(mt_map)

In [ ]:
abs_corr.get_decomposition_loadings().plot(title = "Corrected")
abs.get_decomposition_loadings().plot(title = "absorption")
no_abs.get_decomposition_loadings().plot(title = "no thickness")

In [ ]:
for i,j in enumerate(cropped_spim.metadata.Sample.elements) :
    print("element {}: {}".format(j,est_ref.W_[i])) 

In [ ]:
no_abs = hs.load('eds_rebin_results_no_thickness.hspy')
abs = hs.load('eds_rebin_results_abs.hspy')
abs_corr = hs.load('eds_rebin_results_after_correction.hspy')

In [ ]:
cropped_spim.plot_decomposition_results()

fW = est_aft.W_
print_concentrations_from_W(fW[:-2,:],elements = cropped_spim.metadata.Sample.elements)

In [ ]:
cfW = np.vstack((fW[:5],fW[6:]))
sfw = cfW.sum(axis = 1)
ssfw = sfw/sfw.sum()
nssfw = ssfw
for i,j in enumerate(ns.metadata.Sample.elements) : 
    print('{} : {}'.format(j,nssfw[i]))


In [ ]:
abscr.shape

In [ ]:
plt.plot(np.mean(G()@est.W_@est.H_, axis = 1))
plt.plot(ns.mean())

In [ ]:
ed = {}
i = 0
for elt in cropped_spim.metadata.Sample.elements : 
    if elt == "Fe" : 
        i += 1
    ed[elt] = np.mean(est.W_[i])
    
    i+=1

summ = 0
for i in ed : 
    summ += ed[i]
    
for i in ed : 
    ed[i] = ed[i]/summ
    
ed

In [ ]:
input_filename = str("phyllo_spectre_moyen.hspy")
spectrum = hs.load(input_filename)

spectrum.axes_manager[0].offset += +0.000

spectrum.set_signal_type("EDS_espm")
spectrum.set_analysis_parameters(beam_energy = 300,
azimuth_angle = 0.0,
elevation_angle = 18.0,
elements = ['Al', 'C', 'Ca', 'Cl', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ni', 'O', 'P', 'Pt', 'S', 'Si', 'Ti', 'Zr'],
tilt_stage = 0.001,
thickness = 80e-7,
density = 2.1,

detector_type = "Det_eff.txt",
width_slope = 0.01,
width_intercept = 0.065,
xray_db = "300keV_corentin_xrays.json")
# fW = spim.set_fixed_W({"p0" : {"Si" : 0.0},"p1" : {"Fe" : 0.0}, "p2" : {"Mg" : 0.0}})

G = spectrum.build_G("bremsstrahlung",norm = True)
est = SmoothNMF( n_components = 1,tol=0.000001, max_iter = 1000, G = G)

est.fit_transform(spectrum.data[:,np.newaxis])
fW = est.W_
fG = est.G_

In [ ]:
x = np.linspace(spectrum.axes_manager[0].offset, spectrum.axes_manager[0].offset + spectrum.axes_manager[0].size*spectrum.axes_manager[0].scale,num = spectrum.axes_manager[0].size)
plt.plot(x,spectrum.data,label="exp")
plt.plot(x,fG@fW,label="fit")
plt.legend()

In [ ]:
print_concentrations_from_W(fW[:-2,:],elements = spectrum.metadata.Sample.elements)

In [ ]:
k_names = ["Ag_La","Al_Ka", "Ar_Ka","B_Ka","Bi_Ma","C_Ka","Ca_Ka","Ca_La","Cl_Ka","Cr_Ka",
        "Cr_La","Cu_Ka","Cu_La","F_Ka","Fe_Ka","Fe_La","Fe_Lb3","Fe_Ll","Fe_Ln","Ga_Ka",
        "Ga_La","Ho_La","La_La","K_Ka","Mg_Ka","Mn_Ka","Mn_La","Mg_La","N_Ka","Na_Ka",
        "Ni_Ka","Ni_La","O_Ka","P_Ka","Pb_La","Pt_La","Pt_Ma","S_Ka","Si_Ka","Sn_La",
        "Ti_Ka","Tl_Ma","Zr_Ka","Zr_La","Zn_Ka"]
kfactors = [1,
 1.05,
 1.188,
 5,
 1,
 1.42,
 1.16,
 8.8,
 1.093,
 1.314,
 5.23,
 1.773,
 1.523,
 1,
 1.5,
 2.55,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1.119,
 0.94,
 1.5,
 3.36,
 3.36,
 1.18,
 0.98,
 1.587,
 1.582,
 0.95,
 1.096,
 2.85,
 1,
 1,
 1.18,
 1,
 1,
 1.25,
 1,
 3.65,
 1.49,
 1.96]


In [ ]:
table, mdata = tu.load_table("300keV_xrays.json")
n_table, n_mdata = tu.import_k_factors(table,mdata, k_names, kfactors,  ref_name = "Si_Ka")
tu.save_table("300keV_corentin_xrays.json",n_table,n_mdata)

In [ ]:
didi = {}
for i,j in enumerate(k_names) : 
    didi[j] = kfactors[i]

didi

In [ ]:
a,b = tu.load_table("300keV_corentin_xrays.json")
a['26']["L3M4"]['cs']/a['26']["KL3"]['cs']
a["26"]

In [ ]:
cs_sum1 = 0
cs_sum2 = 0
for i in a["26"] : 
    if i[0] == "K" : 
        cs_sum1 += a["26"][i]['cs']
    if i[0] == "L" : 
        cs_sum2 += a["26"][i]['cs']

In [ ]:
cs_sum2/cs_sum1

In [ ]:
cs_sum1